In [1]:
import findspark
findspark.init("/home/gorazda/spark-2.4.7-bin-hadoop2.7")
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("logistic_regression_titanic").getOrCreate()

In [7]:
data = spark.read.csv("titanic.csv", inferSchema=True, header=True)

In [8]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [11]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [12]:
my_cols = data.select(["Survived", "Pclass", 'Sex',
                       'Age', 'SibSp', 'Parch',
                       'Fare', 'Cabin', 'Embarked'])

In [13]:
my_final_data = my_cols.na.drop()

In [14]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer

In [15]:
gender_indexer = StringIndexer(inputCol="Sex", outputCol="SexIndex")

In [16]:
gender_encoder = OneHotEncoder(inputCol="SexIndex", outputCol="SexVec")

In [17]:
embark_indexer = StringIndexer(inputCol="Embarked", outputCol="EmbarkedIndex")
embark_encoder = OneHotEncoder(inputCol="EmbarkedIndex", outputCol="EmbarkedVec")

In [25]:
assembler = VectorAssembler(inputCols=["Pclass", "SexVec", "EmbarkedVec", 'Age', 'SibSp', 'Parch', 'Fare'],
                           outputCol="features")

In [20]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [21]:
log_reg_titanic = LogisticRegression(featuresCol="features", labelCol="Survived")

In [26]:
pipeline = Pipeline(stages=[gender_indexer, embark_indexer,
                           gender_encoder, embark_encoder,
                           assembler, log_reg_titanic])

In [27]:
train_data, test_data = my_final_data.randomSplit([0.7, 0.3])

In [28]:
fit_model = pipeline.fit(train_data)

In [30]:
results = fit_model.transform(test_data)

In [31]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [36]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Survived")

In [38]:
results.select("Survived", "prediction").show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       1|       1.0|
+--------+----------+
only showing top 20 rows



In [39]:
auc = my_eval.evaluate(results)

In [40]:
auc

0.7228070175438597